In [145]:
# 데이터 관련
import pandas as pd
import numpy as np

#시간 측정용
from tqdm import tqdm

# ----------------------- 한글 ----------------------- #
import MeCab
mec = MeCab.Tagger()

# 점수 게산
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# Tokenizer , Word2Vec
from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec

import collections
import os
import math

import pickle

import datetime
from datetime import timedelta

from collections import Counter

In [ ]:
# ----------------------- 테스트 ----------------------- #

In [143]:
# ------------------------------- Data load ------------------------------- #
beh = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv")
tra = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv")
cus = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv")
cla = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv")

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# 사용할 핫 키워드의 갯수
hkl = 15
# 출력할 카테고리
hca = 5
gdict = {0:"남성",1:"여성",-1:"식별 불가"}
adict = {0:"10/20대",1:"30대",2:"40대",3:"50/60대",-1:"식별 불가"}


In [245]:
# 키워드 입력
keyword = "pepe"
# 현재 시각
date = "20190910"
time = "10:16"
pd_c = 25
pd_c_len = 8
print("PD_C {0} {1} 구매시 추천 카테고리 : ".format(pd_c,cla[cla["pd_c"]==pd_c]["clac_nm3"].values[0]),(Recommend_PD_C(pd_c,pd_c_len)))
print("------------------------------------------------------------------------------------------------------------------")
print("형태소 분석 : ",KoEnMorphs(keyword))
print("------------------------------------------------------------------------------------------------------------------")
print("키워드 관련 대분류 카테고리 출력 : ",Kwd2Category1(keyword))
print("------------------------------------------------------------------------------------------------------------------")
print("남/여 예측 : ",gdict[pred_Gender(keyword)])
print("연령대 예측 : ",adict[pred_Age(keyword)])
print("------------------------------------------------------------------------------------------------------------------")
print("입력 시각",date,time)
print("핫 키워드 관련 : ",Hotkeyword_Category(date,time,hkl,hca))

PD_C 25 Crack Removals 구매시 추천 카테고리 :  ['Car Cleaners', 'Automobile Oil / Additives', 'Car Cleaning Rags', 'Trash Bags', 'Coatings / Waxes', "Women's Hair Accessories", 'Blueberries', "Men's Panties"]
------------------------------------------------------------------------------------------------------------------
형태소 분석 :  ['pepe']
------------------------------------------------------------------------------------------------------------------
키워드 관련 대분류 카테고리 출력 :  ["Women's Clothing"]
------------------------------------------------------------------------------------------------------------------
남/여 예측 :  식별 불가
연령대 예측 :  식별 불가
------------------------------------------------------------------------------------------------------------------
입력 시각 20190910 10:16
핫 키워드 관련 :  ['Vegetables', 'Underwear / Socks and Hosiery / Homewear', 'Fruits', 'Meats', 'Canned / Jarred Foods']


In [ ]:
# ----------------------- 테스트 ----------------------- #

In [132]:
# ----------------------- 정규표현식 ----------------------- #
import re
is_ProductName = re.compile("[a-zA-Z0-9]+")
# ----------------------- 형태소 분석 ----------------------- #
def MCSplit(keyword):
    ret = []
    for i in keyword.split("\n"):
        app = i.split("\t")[0]
        if app != 'EOS':
            ret.append(app)
        else:
            return ret
    return ret
def subSChar(keyword):
    ret = re.sub("[-=+,.!@#$%^&*(\)[\]<\>{\}\'\"/_|ㄱ-ㅎ|ㅏ-ㅣ]"," ",keyword)
    return ret
def KoEnMorphs(keyword):
    x = (subSChar(keyword.lower())).split()
    for i in range(0,len(x)):
        for j in is_ProductName.findall(x[i]):
            x[i] = x[i].replace(j," " + j + " ")
    x = (" ".join(x)).split()
    ret = []
    for i in x:
        if is_ProductName.fullmatch(i) == None:
            ret += MCSplit(mec.parse((i)))
        else:
            ret.append(i)
    return ret

In [232]:
# Load Tokenizer
with open('./Tokenizer/Tokenizer1020.pickle', 'rb') as handle:
    tok1020 = pickle.load(handle)
with open('./Tokenizer/Tokenizer30.pickle', 'rb') as handle:
    tok30 = pickle.load(handle)
with open('./Tokenizer/Tokenizer40.pickle', 'rb') as handle:
    tok40 = pickle.load(handle)
with open('./Tokenizer/Tokenizer5060.pickle', 'rb') as handle:
    tok5060 = pickle.load(handle)

with open('./Tokenizer/TokenizerF.pickle', 'rb') as handle:
    tokF = pickle.load(handle)
with open('./Tokenizer/TokenizerM.pickle', 'rb') as handle:
    tokM = pickle.load(handle)
    
# Load Word2Vec
wv1020 = Word2Vec.load("./Word2Vec/Word2Vec_1020.model")
wv30  = Word2Vec.load("./Word2Vec/Word2Vec_30.model")
wv40 = Word2Vec.load("./Word2Vec/Word2Vec_40.model")
wv5060 = Word2Vec.load("./Word2Vec/Word2Vec_5060.model")
wvF = Word2Vec.load("./Word2Vec/Word2Vec_Female.model")
wvM = Word2Vec.load("./Word2Vec/Word2Vec_Male.model")

# Load 대분류 토크나이저 
Tokenizer_Clac1 = []
for i in range(0,60):
    with open('./Tokenizer/Clac_nm/Tokenizer_Clac1_{0}.pickle'.format(i), 'rb') as handle:
        Tokenizer_Clac1.append(pickle.load(handle))
# Load 중분류 토크나이저
Tokenizer_Clac2 = []
for i in range(0,332):
    with open('./Tokenizer/Clac_nm/Tokenizer_Clac2_{0}.pickle'.format(i), 'rb') as handle:
        Tokenizer_Clac2.append(pickle.load(handle))

# 중,대분류 이름
clac_nm1_Name = np.load("./Keyword in Clac_nm123/대분류 pd_c Name 파일.npy")
clac_nm2_Name = np.load("./Keyword in Clac_nm123/중분류 pd_c Name 파일.npy")

In [134]:
# 게산식
def CalcPred(x1,x2,sim):
    return math.sqrt(1/x1 + 1/x2 + sim)

In [135]:
# ----------------------------------------- 성별 예측 ----------------------------------------- #
def pred_Gender(keyword):
    predF = 0
    predM = 0
    morphs = KoEnMorphs(keyword)
    lenF = len(tokF.word_index)
    lenM = len(tokM.word_index)

    list_Fkwd = []
    list_Mkwd = []
    
    simFN = 0
    simMN = 0
    # 검색 키워드 한개일때
    if len(morphs) == 1:
        scoreF = tokF.texts_to_sequences(morphs)
        if len(scoreF[0]) == 0:
            predF = 0
        else:
            predF = (1/scoreF[0][0])
            
        scoreM = tokM.texts_to_sequences(morphs)
        if len(scoreM[0]) == 0:
            predM = 0
        else:
            predM = (1/scoreM[0][0])
    else:
        # 키워드가 2개이상 일때
        # Tokenize 화
        scoreF = tokF.texts_to_sequences(morphs)
        scoreM = tokM.texts_to_sequences(morphs)

        # Tokenizer 에 존재하지 않아 빈 리스트 []생성시 제거 
        while(1):
            try:
                del scoreF[scoreF.index([])]
            except:
                break
        while(1):
            try:
                del scoreM[scoreM.index([])]
            except:
                break

        # 여성 모델 계산
        if len(scoreF) == 0:
            predF = 0
        else:
            for i in range(0,len(scoreF)):
                if i == 0:
                    continue
                # x1, x2 순위
                x1 =  (scoreF[i-1][0])
                x2 =  (scoreF[i][0])
                try:
                    # 유사도 키워드 x1 , x2 의 유사도
                    simF = wvF.wv.n_similarity( tokF.sequences_to_texts([scoreF[i-1]]) , tokF.sequences_to_texts([scoreF[i]]))
                    predF += CalcPred(x1,x2,simF)
                    simFN += 1
                except:
                    # 유사도가 존재하지 않을시 
                    simFN += 0
                    
        # 남성 모델 계산
        if len(scoreM) == 0:
            predM = 0
        else:
            for i in range(0,len(scoreM)):
                if i == 0:
                    continue
                x1 = (scoreM[i-1][0])
                x2 = (scoreM[i][0])
                try:
                    simM = wvM.wv.n_similarity( tokM.sequences_to_texts([scoreM[i-1]]) , tokM.sequences_to_texts([scoreM[i]]))
                    predM += CalcPred(x1,x2,simM)
                    simMN += 1
                except:
                    simMN += 0
                    
    # 평균을 구한다.
    if simFN != 0:
        predF /= simFN
    if simMN != 0:
        predM /= simMN
        
    if predM > predF:
        return 0
    elif predF > predM:
        return 1
    else:
        return -1

In [136]:
# ----------------------------------------- 나이 예측 ----------------------------------------- #
def pred_Age(keyword):
    pred1020 = 0
    pred30 = 0
    pred40 = 0
    pred5060 = 0
    
    morphs = KoEnMorphs(keyword)
    
    len1020 = len(tok1020.word_index)
    len30 = len(tok30.word_index)
    len40 = len(tok40.word_index)
    len5060 = len(tok5060.word_index)
    
    list_1020kwd = []
    list_30kwd = []
    list_40kwd = []
    list_5060kwd = []
    
    sim1020N = 0
    sim30N = 0
    sim40N = 0
    sim5060N = 0
    
    # 검색 키워드 한개일때
    if len(morphs) == 1:
        
        score1020 = tok1020.texts_to_sequences(morphs)
        if len(score1020[0]) == 0:
            pred1020 = 0
        else:
            pred1020 = (1/score1020[0][0])
            
        score30 = tok30.texts_to_sequences(morphs)
        if len(score30[0]) == 0:
            pred30 = 0
        else:
            pred30 = (1/score30[0][0])
            
        score40 = tok40.texts_to_sequences(morphs)
        if len(score40[0]) == 0:
            pred40 = 0
        else:
            pred40 = (1/score40[0][0])
            
        score5060 = tok5060.texts_to_sequences(morphs)
        if len(score5060[0]) == 0:
            pred5060 = 0
        else:
            pred5060 = (1/score5060[0][0])
    else:
        
        score1020 = tok1020.texts_to_sequences(morphs)
        score30 = tok30.texts_to_sequences(morphs)
        score40 = tok40.texts_to_sequences(morphs)
        score5060 = tok5060.texts_to_sequences(morphs)

        # 빈 배열 삭제
        while(1):
            try:
                del score1020[score1020.index([])]
            except:
                break
        while(1):
            try:
                del score30[score30.index([])]
            except:
                break
        while(1):
            try:
                del score40[score40.index([])]
            except:
                break
        while(1):
            try:
                del score5060[score5060.index([])]
            except:
                break

        if len(score1020) == 0:
            pred1020 = 0
        else:
            for i in range(0,len(score1020)):
                if i == 0:
                    continue
                x1 = (score1020[i-1][0])
                x2 = (score1020[i][0])
                try:
                    sim1020 = wv1020.wv.n_similarity(tok1020.sequences_to_texts([score1020[i-1]]),tok1020.sequences_to_texts([score1020[i]]))
                    pred1020 += CalcPred(x1,x2,sim1020)
                    sim1020N += 1
                except:
                    continue
                    
        if len(score30) == 0:
            pred30 = 0
        else:
            for i in range(0,len(score30)):
                if i == 0:
                    continue
                x1 = (score30[i-1][0])
                x2 = (score30[i][0])
                try:
                    sim30 = wv30.wv.n_similarity( tok30.sequences_to_texts([score30[i-1]]) , tok30.sequences_to_texts([score30[i]]))
                    pred30 += CalcPred(x1,x2,sim30)
                    sim30N += 1
                except:
                    continue
                    
        if len(score40) == 0:
            pred40 = 0
        else:
            for i in range(0,len(score40)):
                if i == 0:
                    continue
                x1 = (score40[i-1][0])
                x2 = (score40[i][0])
                try:
                    sim40 = wv40.wv.n_similarity(tok40.sequences_to_texts([score40[i-1]]),tok40.sequences_to_texts([score40[i]]))
                    pred40 += CalcPred(x1,x2,sim40)
                    sim40N += 1
                except:
                    continue
                    
        if len(score5060) == 0:
            pred5060 = 0
        else:
            for i in range(0,len(score5060)):
                if i == 0:
                    continue
                x1 = (score5060[i-1][0])
                x2 = (score5060[i][0])
                try:
                    sim5060 = wv5060.wv.n_similarity( tok5060.sequences_to_texts([score5060[i-1]]) , tok5060.sequences_to_texts([score5060[i]]))
                    pred5060 += CalcPred(x1,x2,sim5060)
                    sim5060N += 1
                except:
                    continue
                    
    if sim1020N != 0:
        pred1020 /= sim1020N
    if sim30N != 0:
        pred30 /= sim30N
    if sim40N != 0:
        pred40 /= sim40N
    if sim5060N != 0:
        pred5060 /= sim5060N
    ret = [pred1020,pred30,pred40,pred5060]
    
    if (pred1020 == pred30) and (pred1020 == pred40) and (pred1020 == pred5060):
        return -1
    else:
        return ret.index(max(ret))

In [179]:
def Hotkeyword_Category(date,time,hkwdLen,categorylen):
    #입력 시간을 기준으로 4시간동안 수집한 많이 검색된 키워드를 추천한다.
    now = datetime.datetime.strptime(str(date)+time,"%Y%m%d%H:%M")
    dt = datetime.datetime.strptime(str(date)+time,"%Y%m%d%H:%M") - timedelta(hours=4)
    kwd = []
    # 조건에 따라 키워드를 저장
    for i in beh[((beh["sess_dt"]==int(str(dt.date()).replace("-","")))&(beh["hit_tm"]>=str(dt.time())[0:5])&(beh["action_type"]==0))|((beh["sess_dt"]==int(str(now.date()).replace("-","")))&(beh["hit_tm"]<=str(now.time())[0:5])&(beh["action_type"]==0))]["sech_kwd"].values:
        for j in KoEnMorphs(i):
            kwd.append(j)
    # 저장한 키워드들을 Tokenizer
    tokk = Tokenizer(lower=True)
    tokk.fit_on_texts(kwd)
    # 가장 많이 검색한 20개의 키워드를 추출
    hotKwd = [tokk.index_word[i] for i in range(1,hkwdLen)]
    ret_Category = []
    for i in hotKwd:
        ret_Category += Kwd2Category1(i)
    # 키워드들을 카테고리화
    return [key for key, _ in Counter(ret_Category).most_common(categorylen)]

In [137]:
# 계산식
def SequenceCalc(sequence,tok):
    ret = 0
    for i in sequence:
        if len(i) != 0:
            # 순위가 높을수록 해당 카테고리일 확률이 높음
            # 1/순위 * (단어가 나온 횟수 / 해당 문서의 총 길이)
            ret += (1/i[0])*(tok.word_counts[tok.index_word[i[0]]]/tok.document_count)
        else:
            ret += 0.0
    return ret
# 대분류 출력
def Kwd2Category1(keyword):
    temp = []
    # 순위 계산
    for tok in Tokenizer_Clac1:
        temp.append(SequenceCalc(tok.texts_to_sequences(KoEnMorphs(keyword)),tok))
    # 정렬해준다.
    tempSrt = sorted(temp, key = lambda x:float(x))
    
    index = []
    for i in reversed(range(0,len(tempSrt))):
        if tempSrt[i] != 0:
            index.append(temp.index(tempSrt[i]))
    # 인덱스를 찾아준다.
    ret = []
    for i in index:
        ret.append(clac_nm1_Name[i])
    # 상위 5개만 출력
    return ret[:5]

# 중분류 출력 
def Kwd2Category2(keyword):
    temp = []
    # 순위 계산
    for tok in Tokenizer_Clac2:
        temp.append(SequenceCalc(tok.texts_to_sequences(KoEnMorphs(keyword)),tok))
    # 정렬해준다.
    tempSrt = sorted(temp, key = lambda x:float(x))
    
    index = []
    for i in reversed(range(0,len(tempSrt))):
        if tempSrt[i] != 0:
            index.append(temp.index(tempSrt[i]))
    # 인덱스를 찾아준다.
    ret = []
    for i in index:
        ret.append(clac_nm2_Name[i])
    return ret[:5]

In [234]:
# PD_C 추천
pd_cArr = np.load("./Keyword in Clac_nm123/pd_cArr.npy")
def Recommend_PD_C(pdc,categorylen):
    if len(Counter(pd_cArr[pdc-1])) <= categorylen:
        categorylen = len(Counter(pd_cArr[pdc-1]))
        if categorylen == 0:
            print("추천 할 수 있는 데이터가 없습니다.")
            return -1
    return [key for key, _ in Counter(pd_cArr[pdc-1]).most_common(categorylen)]